In [1]:
import numpy as np

In [27]:
vectors = np.random.rand(300000, 150)

In [28]:
vector = np.random.rand(1, 150)

In [29]:
vectors.nbytes

360000000

In [30]:
np.save('test.npy', vectors)

In [31]:
%%timeit

vectors = np.load('test.npy')

168 ms ± 910 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
import pickle

In [8]:
with open('test.pkl', 'wb') as out_file:
    pickle.dump(vectors, out_file)

In [9]:
%%timeit

with open('test.pkl', 'rb') as out_file:
    vectors = pickle.load(out_file)

232 ms ± 6.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
products = ['cloud_jira', 'cloud_misc', 'cloud_confluence', 'hipchat', 'stride', 'cloud_bitbucket']

In [11]:
sources = ['jac', 'cac', 'gsac', 'community', 'fd']

In [12]:
index = {}
for product in products:
    for source in sources:
        index[product + ':' + source + ':' + 'docs'] = np.random.randint(1000000, size=10000)
        index[product + ':' + source + ':' + 'vectors'] = np.random.rand(10000, 150)

In [13]:
np.savez('multiple.npz', **index)

In [22]:
%%timeit

index = None
with np.load('multiple.npz') as index_file:
    index = dict(index_file)

253 ms ± 3.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
with open('multiple.pkl', 'wb') as out_file:
    pickle.dump(index, out_file)

In [16]:
%%timeit

with open('multiple.pkl', 'rb') as out_file:
    index = pickle.load(out_file)

227 ms ± 14.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
def closest(vectors, vector):
    diff = vectors - vector
    ranks = (diff * diff).sum(axis=1)
    return np.argsort(ranks)

In [18]:
def search_docs(docs, vectors, vector):
    idx = closest(vectors, vector)
    return docs[idx]

In [19]:
def search_index(index, query):
    docs = []
    for product in query['products']:
        for source in query['sources']:
            docs_key = product + ':' + source + ':docs'
            vectors_key = product + ':' + source + ':vectors'
            if docs_key in index:
                candidate_docs = index[docs_key]
                vectors = index[vectors_key]
                found_docs = search_docs(candidate_docs, vectors, query['vector'])
                docs.extend(found_docs)
    return docs

In [20]:
query = {
    'products': ['cloud_jira', 'cloud_misc'],
    'sources': ['jac', 'cac', 'gsac', 'community', 'fd'],
    'vector': np.random.rand(150)
}

In [23]:
%%timeit

search_index(index, query)

58.1 ms ± 431 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%%timeit

vectors2 = vectors.copy()

191 ms ± 529 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
vectors

array([[3.55148564e-01, 4.64306397e-01, 7.61114490e-01, ...,
        1.16801514e-01, 5.84080492e-01, 3.38019641e-01],
       [2.64657901e-01, 8.10657240e-01, 6.72680205e-01, ...,
        6.83145768e-02, 4.94839718e-01, 6.07834048e-01],
       [7.51468218e-01, 9.33274884e-01, 9.14571393e-01, ...,
        7.97104327e-01, 7.05785688e-01, 9.59173189e-01],
       ...,
       [7.03779348e-04, 8.05915299e-01, 1.99763365e-01, ...,
        8.31354990e-01, 8.93746551e-01, 7.30650320e-01],
       [3.33047567e-01, 7.15124904e-02, 9.70056551e-01, ...,
        8.46879143e-02, 5.50589253e-01, 7.55409485e-01],
       [9.30991460e-01, 2.62388989e-01, 6.79466549e-01, ...,
        7.77976840e-01, 1.57419637e-02, 4.52495642e-02]])